In [2]:
import numpy as np
from numpy.linalg import lstsq, norm

# Set the seed for reproducibility
np.random.seed(1234)

mu = 0
sigma = 1
theta = 1

Xzero = 1

T = 1
N = 2**11
dt = T/N

reps = 10000
decreasing_number = 6

dW = np.sqrt(dt) * np.random.randn(reps, N)
W = np.cumsum(dW, axis=1)

Xem = {}

for i in range(1, decreasing_number + 1):
    Xem[i - 1] = np.zeros((reps, N//2**(i-1)+1))

Xpath = np.zeros((reps, N+1))
Xpath[:, 0] = Xzero

integral = np.exp(theta * np.arange(0, T, dt)).reshape(1, -1) * dW
Xpath[:, 1:] = (Xzero * np.exp(-theta * np.arange(dt, T+dt, dt)).reshape(1, -1) +
                mu * (1 - np.exp(-theta * np.arange(dt, T+dt, dt)).reshape(1, -1)) +
                sigma * np.cumsum(integral, axis=1) * np.exp(-theta * np.arange(dt, T+dt, dt)).reshape(1, -1))

Xpath0 = np.zeros((reps, N+1))
Xpath0[:, 0] = Xzero
for i in range(N):
    Xpath0[:, i + 1] = (Xpath0[:, i] * np.exp(-theta*dt) +
                        mu * (1 - np.exp(-theta*dt)) +
                        sigma * np.sqrt((1 - np.exp(-2*theta*dt))/(2*theta)) * dW[:, i] / np.sqrt(dt))

for i in range(1, decreasing_number + 1):
    Xem[i - 1][:, 0] = Xzero

    dWTemp = np.zeros((reps, N//2**(i-1)))

    for j in range(2**(i-1)):
        dWTemp += dW[:, j::2**(i-1)]

    for j in range(N//2**(i-1)):
        Xem[i - 1][:, j + 1] = (Xem[i - 1][:, j] +
                                theta * (mu - Xem[i - 1][:, j]) * dt * 2**(i-1) +
                                sigma * dWTemp[:, j])

expectedError = np.zeros(decreasing_number)
expectedError0 = np.zeros(decreasing_number)
for i in range(1, decreasing_number + 1):
    error = np.abs(Xem[i - 1][:, -1] - Xpath[:, -1])
    expectedError[i-1] = np.mean(error)

    error0 = np.abs(Xem[i - 1][:, -1] - Xpath0[:, -1])
    expectedError0[i-1] = np.mean(error0)

log2_expectedError = np.log2(expectedError)
log2_expectedError0 = np.log2(expectedError0)

A = np.column_stack((np.ones(decreasing_number), np.arange(int(np.log2(dt)), decreasing_number + int(np.log2(dt)))))
b = log2_expectedError
b_0 = log2_expectedError0

x, _, _, _ = lstsq(A, b, rcond=None)
x_0, _, _, _ = lstsq(A, b_0, rcond=None)

print("The slope (cumsum integral) is ", x[1])
print("The intercept (cumsum integral) is ", x[0])
print("The residual (cumsum integral) is ", norm(A @ x - b))
print("The slope (loop integral) is ", x_0[1])
print("The intercept (loop integral) is ", x_0[0])
print("The residual (loop integral) is ", norm(A @ x_0 - b_0))

The slope (cumsum integral) is  0.87279055620581
The intercept (cumsum integral) is  -2.65638584339459
The residual (cumsum integral) is  0.22401452778288364
The slope (loop integral) is  1.0397374810985112
The intercept (loop integral) is  -1.5300595190550104
The residual (loop integral) is  0.13774735475476033
